# Preprocessor
This module contains the additional nbconvert preprocessors that are required by Literary.

In [1]:
%load_ext literary.notebook

In [2]:
from nbconvert.preprocessors import Preprocessor
from traitlets import Set, Unicode

The Literary exporter uses an "allow-list" model to determine which cells can be exported. By default, no cells are exported to the final script.

In [3]:
class TagAllowListPreprocessor(Preprocessor):
    allow_cell_tags = Set(Unicode(), default_value={"export", "docstring"})

Let's implement a conditional function that determines whether a cell is a member of the allowlist. We just check whether it has any of the `allow_cell_tags` tags.

In [3]:
@patch(TagAllowListPreprocessor)
def check_cell_conditions(self, cell, resources: dict, index: int) -> bool:
    tags = cell.metadata.get("tags", [])
    return bool(self.allow_cell_tags.intersection(tags))

The `preprocess` function modifies the original notebook to eliminate any cells which do not meet the above condition

In [3]:
@patch(TagAllowListPreprocessor)
def preprocess(self, nb, resources: dict):
    nb.cells = [
        self.preprocess_cell(cell, resources, i)[0]
        for i, cell in enumerate(nb.cells)
        if self.check_cell_conditions(cell, resources, i)
    ]
    return nb, resources

For completeness, we define a preprocessor function that subclasses *could* override if they saw fit.

In [3]:
@patch(TagAllowListPreprocessor)
def preprocess_cell(self, cell, resources: dict, index: int):
    return cell, resources